In [50]:
import requests
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from stop_words import get_stop_words

In [69]:
chara = ['hero', 'claudio', 'beatrice', 'leonato', 'don', 'pedro', 'john','benedick', 'margaret', 
         'borachio', 'conrad', 'dogberry', 'verges', 'antonio', 'balthasar', 'ursula', 'much', 'nothing' ]

In [70]:
def key_df(content, chara): 
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    token = tokenizer.tokenize(content)
    stop_words = list(get_stop_words('en'))         
    nltk_words = list(stopwords.words('english')) 
    stop_words.extend(nltk_words)
    keywords = [word.lower() for word in token if not word.lower() in stop_words]
    keywords = [word.lower() for word in keywords if not word in chara]
    key_df = pd.DataFrame(keywords, columns = ['word'])
    key_df = key_df['word'].value_counts()
    key_df = key_df.to_frame()
    
    return key_df

In [71]:
def sparknotes(url):
    html = urlopen(url)
    bs = BeautifulSoup(html.read(), 'html.parser')
    title = bs.find('h1', {'class':"TitleHeader_title tag--titleTitle-notLink"}).get_text()
    content = bs.find('div', {'class':"studyGuideText hack-to-hide-first-h2"}).get_text()
    
    return content

url = 'https://www.sparknotes.com/shakespeare/muchado/summary/'
spark = sparknotes(url)

In [72]:
def shmoop(url):
    html = urlopen(url)
    bs = BeautifulSoup(html.read(), 'html.parser')
    title = bs.find('h2', {'class':"title"}).get_text()
    content = bs.find('div', {'data-class':"SHPlotOverviewSection"}).get_text()
    
    return content

url = 'https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/summary'
shmoop = shmoop(url)

In [73]:
def cliffs(url):
    html = urlopen(url)
    bs = BeautifulSoup(html.read(), 'html.parser')
    title = bs.find('h1').text
    content = bs.find_all("p",{"class":"litNoteText"})
    content = '\n'.join([c.text for c in content])
    
    return content

url = 'https://www.cliffsnotes.com/literature/m/much-ado-about-nothing/play-summary'
cliffs = cliffs(url)

In [74]:
def sparknotes_themes(url):
    html = urlopen(url)
    bs = BeautifulSoup(html.read(), 'html.parser')
    content = bs.findAll('div',attrs={"class":"studyGuideText hack-to-hide-first-h2"})
    for x in content:
        content = (x.find_all(['p','h3']))
    content = '\n'.join([c.text for c in content])
    
    return content

url_1 = "https://www.sparknotes.com/shakespeare/muchado/themes/"
url_2 = "https://www.sparknotes.com/shakespeare/muchado/themes/page/2/"
spark_theme_1 = sparknotes_themes(url_1)
spark_theme_2 = sparknotes_themes(url_2)

In [75]:
def shmoop_themes(url):
    html = urlopen(url)
    bs = BeautifulSoup(html.read(), 'html.parser')
    content = bs.find_all("div", {"data-element":"collapse_target"})
    for x in content:
        content = (x.find_all("p"))
    content = '\n'.join([c.text for c in content])
    
    return content 

url_1 = "https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/themes/marriage"
url_2 = "https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/themes/lies-and-deceit"
url_3 = "https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/themes/language-and-communication"
url_4 = "https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/themes/love"
url_5 = "https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/themes/respect-and-reputation"
url_6 = "https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/themes/transformation"
url_7 = "https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/themes/gender"
url_8 = "https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/themes/pride"
url_9 = "https://www.shmoop.com/study-guides/literature/much-ado-about-nothing/themes/maturity"
shmoop_theme_1 = shmoop_themes(url_1)
shmoop_theme_2 = shmoop_themes(url_2)
shmoop_theme_3 = shmoop_themes(url_3)
shmoop_theme_4 = shmoop_themes(url_4)
shmoop_theme_5 = shmoop_themes(url_5)
shmoop_theme_6 = shmoop_themes(url_6)
shmoop_theme_7 = shmoop_themes(url_7)
shmoop_theme_8 = shmoop_themes(url_8)
shmoop_theme_9 = shmoop_themes(url_9)

In [76]:
def utah(url):
    html = urlopen(url)
    bs = BeautifulSoup(html.read(), 'html.parser')
    content = bs.find_all("div", {"class":"sqs-block-content"})
    for x in content:
        content = (x.find_all("p"))
    content = '\n'.join([c.text for c in content])
    
    return content

url = "https://www.bard.org/study-guides/synopsis-much-ado-about-nothing"
utah = utah(url)

In [77]:
contents = [spark, shmoop, cliffs, spark_theme_1, spark_theme_2, 
            shmoop_theme_1, shmoop_theme_2, shmoop_theme_3, shmoop_theme_4, shmoop_theme_5, 
            shmoop_theme_6, shmoop_theme_7, shmoop_theme_8, shmoop_theme_9, utah]

keys_df = pd.DataFrame()
for content in contents:
    df = key_df(content, chara)
    keys_df = keys_df.append(df)

    


In [79]:
keys_df = keys_df.groupby([keys_df.index]).sum().sort_values(by = 'word', ascending = False)

In [82]:
keys_df.to_csv("maan.csv")